In [1]:
import data
import langid
import importlib
importlib.reload(data)

<module 'data' from 'C:\\Users\\katha\\Documents\\GitHub\\low-resource-machine-translation\\Data_clean_skripts\\data.py'>

In [2]:
TA_BLACKLIST = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz-(\')/[]♪/%#$&\/_"{.}|=<>@~`'
EN_BLACKLIST = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ"#""$%&\()*+-./:;<=>@[\\]^_`♪{|}~='

In [3]:
def readfile(filename):
    return open(filename,encoding='utf-8').read().split('\n')[:-1]

# Loading raw data

In [4]:
k = 'WMT_Dev'

In [6]:
en_sent=[]
for i in range(1,6):
    file=readfile(f'../Corpus/{k}/newsdev2020-enta-src.en.sgm')
    #file=readfile(f'ta_en_pairs/{k}/english.txt')
    #file=readfile(f'Corpus/{k}/{k}_endata.txt')

    print(len(file))
    for i in range(len(file)):
        en_sent.append(file[i])

2559
2559
2559
2559
2559


In [10]:
en_sent[1]

'<doc sysid="ref" docid="seattle_times.7729" genre="news" origlang="en">'

In [8]:
import re
en_sen=[]
for i in range(len(en_sent)):
    string = en_sent[i]
    sent = re.sub('<.*?>', '', string)
    en_sen.append(sent)

In [ ]:
en_sen[10]

In [ ]:
ta_sent=[]
for i in range(1,6):
    #file=readfile(f'Corpus/{k}/{k}_tadata.txt')
    file=readfile(f'../Corpus/{k}/newsdev2020-enta-ref.ta.sgm')
    #file=readfile(f'ta_en_pairs/{k}/tamil.txt')
    for i in range(len(file)):
        ta_sent.append(file[i])

In [ ]:
ta_sent[10]

In [ ]:
ta_sen=[]
for i in range(len(ta_sent)):
    string = ta_sent[i]
    sent = re.sub('<.*?>', '', string)
    ta_sen.append(sent)

In [ ]:
ta_sen[19]

# Clean and filter 

In [ ]:
en_lines = [ line.lower() for line in en_sen]

ta_lines = [ data.filter_line(line, TA_BLACKLIST) for line in ta_sen ]
en_lines = [ data.filter_line(line, EN_BLACKLIST) for line in en_lines ]

ta_lines = [ " ".join(line.split())+' .' for line in ta_lines ]
en_lines = [ " ".join(line.split())+' .' for line in en_lines ]

In [ ]:
len(en_lines)

In [ ]:
ta_lines, en_lines = data.filter_data(ta_lines, en_lines)

In [ ]:
len(en_lines)

In [ ]:
def save_clean_sentences(sentences1,sentences2,filename1,filename2):
    file1 = open(filename1, 'w',encoding='utf-8')
    file2 = open(filename2, 'w',encoding='utf-8')
    for i,j in zip(sentences1,sentences2):
        file1.write("%s\n" % i)
        file2.write("%s\n" % j)

In [ ]:
save_clean_sentences(en_lines,ta_lines,f'../Corpus/{k}/{k}_endata.txt',f'../Corpus/{k}/{k}_tadata.txt')

In [ ]:
file1=readfile(f'../Corpus/{k}/{k}_endata.txt')
file2=readfile(f'../Corpus/{k}/{k}_tadata.txt')

# Removing Duplicates

In [ ]:
mix=[]
for i,j in zip(file1,file2):
    mix.append(i+"*|*"+j)

In [ ]:
len(mix)


In [ ]:
mix=list(set(mix))

In [ ]:
len(mix)

In [ ]:
en_data=[]
ta_data=[]
for i in mix:
    i=i.split("*|*")
    en_data.append(i[0])
    ta_data.append(i[1])

In [ ]:
mix[50]

## Removing same source with different translation and same translation with different source

In [ ]:
from collections import Counter

In [ ]:
a=Counter(en_data)

In [ ]:
remove=[]
for i,j in a.items():
    if j>1:
        remove.append(i)

In [ ]:
en_data2=[]
ta_data2=[]
for i in range(len(en_data)):
    if en_data[i] not in remove:
        en_data2.append(en_data[i])
        ta_data2.append(ta_data[i])

In [ ]:
len(en_data2)

In [ ]:
b=Counter(ta_data2)

In [ ]:
remove2=[]
for i,j in b.items():
    if j>1:
        remove2.append(i)

In [ ]:
len(remove2)

In [ ]:
en_data3=[]
ta_data3=[]
for i in range(len(ta_data2)):
    if ta_data2[i] not in remove2:
        en_data3.append(en_data2[i])
        ta_data3.append(ta_data2[i])

In [ ]:
len(en_data3)

In [ ]:
save_clean_sentences(en_data3,ta_data3,f'../Corpus/00_Single_Clean_unsplitted_Corpus/{k}_endata_unique.txt',f'../Corpus/00_Single_Clean_unsplitted_Corpus/{k}_tadata_unique.txt')

## Adjusting remaining puctuation

In [ ]:
new1=[]
new2=[]
for i in ta_data3:
    i=i.replace(',',' ,')
    i=i.replace('!',' !')
    i=i.replace('?',' ?')
    new1.append(i)
for i in en_data3:
    i=i.replace(',',' ,')
    i=i.replace('!',' !')
    i=i.replace('?',' ?')
    new2.append(i)
ta_data3=new1
en_data3=new2

In [ ]:
ta_data3[0:10]

In [ ]:
en_data3[0:10]

In [ ]:
len(en_data3)

## Divide and save

In [ ]:
def percentage(percent, whole):
    return (percent * whole) / 100.0

In [ ]:
percent = 70
percent_val = 85
boundry = percentage(percent,len(en_data3))
boundry_train = round(boundry)

oundry = percentage(percent_val,len(en_data3))
boundry_val = round(oundry)


In [ ]:
en_train=en_data3[0:boundry_train]
ta_train=ta_data3[0:boundry_train]
len(en_train)

In [ ]:
en_val=en_data3[boundry_train:boundry_val]
ta_val=ta_data3[boundry_train:boundry_val]
len(en_val)

In [ ]:
en_test=en_data3[boundry_val:]
ta_test=ta_data3[boundry_val:]
len(en_test)

In [ ]:
save_clean_sentences(en_train,ta_train,f'../Corpus/01_Single_Train_Set/{k}_train.en',f'../Corpus/01_Single_Train_Set/{k}_train.ta')
save_clean_sentences(en_test,ta_test,f'../Corpus/03_Single_Test_Sets/{k}_test.en',f'../Corpus/03_Single_Test_Sets/{k}_test.ta')
save_clean_sentences(en_val,ta_val,f'../Corpus/02_Single_Validation_Sets/{k}_val.en',f'../Corpus/02_Single_Validation_Sets/{k}_val.ta')

## Check language

In [ ]:
langid.classify("Hallo, wie geht es dir?")